In [1]:
!pip install flask transformers openai-whisper googletrans==4.0.0-rc1
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!apt install ffmpeg -y


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import threading
from flask import Flask, request, render_template_string
import whisper
from transformers import pipeline
from googletrans import Translator
from google.colab import output

# Load models
whisper_model = whisper.load_model("base")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
translator = Translator()

app = Flask(__name__)
stored_transcript = ""
stored_summary = ""

# Styled HTML Template
HTML = """
<!doctype html>
<html>
<head>
  <title>MeetMate: Smart Meeting Assistant</title>
  <style>
    body {
      font-family: 'Segoe UI', sans-serif;
      background: #f4f7f9;
      color: #333;
      padding: 30px;
      max-width: 900px;
      margin: auto;
    }
    h2, h3 {
      color: #2c3e50;
    }
    form, .output-box {
      background: #fff;
      padding: 20px;
      border-radius: 8px;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
      margin-bottom: 30px;
    }
    input[type="file"], input[type="text"], select {
      padding: 10px;
      margin-top: 10px;
      width: 100%;
      box-sizing: border-box;
    }
    input[type="submit"] {
      background-color: #3498db;
      color: white;
      padding: 10px 15px;
      border: none;
      border-radius: 4px;
      margin-top: 10px;
      cursor: pointer;
    }
    input[type="submit"]:hover {
      background-color: #2980b9;
    }
    pre {
      white-space: pre-wrap;
      word-wrap: break-word;
    }
  </style>
</head>
<body>
  <h2>📋 MeetMate: Smart Meeting Assistant</h2>

  <form method=post enctype=multipart/form-data>
    <label>🎧 Upload a Meeting Audio File (.mp3/.wav):</label><br>
    <input type="file" name="audio" accept="audio/*" required><br>
    <input type="submit" value="Transcribe & Summarize">
  </form>

  {% if transcript %}
  <div class="output-box">
    <h3>📝 Transcript:</h3>
    <pre>{{ transcript }}</pre>

    <h3>📌 Summary:</h3>
    <p>{{ summary }}</p>
  </div>

  <form method="post">
    <h3>🔍 Ask a Question:</h3>
    <input type="text" name="question" placeholder="What was discussed about the budget?" required>
    <input type="submit" value="Get Answer">
  </form>

  {% if answer %}
    <div class="output-box">
      <h3>🤖 Answer:</h3>
      <p>{{ answer }}</p>
    </div>
  {% endif %}

  <form method="post">
    <h3>🌐 Translate Summary:</h3>
    <select name="language">
      <option value="hi">Hindi</option>
      <option value="ta">Tamil</option>
      <option value="fr">French</option>
      <option value="es">Spanish</option>
    </select>
    <input type="submit" value="Translate">
  </form>

  {% if translated %}
    <div class="output-box">
      <h3>🌎 Translated Summary:</h3>
      <p>{{ translated }}</p>
    </div>
  {% endif %}
  {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def index():
    global stored_transcript, stored_summary
    transcript = summary = answer = translated = None

    # Audio upload
    if 'audio' in request.files:
        audio = request.files['audio']
        ext = audio.filename.rsplit('.', 1)[-1]
        audio_path = f"temp_audio.{ext}"
        audio.save(audio_path)

        result = whisper_model.transcribe(audio_path)
        transcript = result["text"]
        stored_transcript = transcript

        summary_output = summarizer(transcript, max_length=150, min_length=40, do_sample=False)
        summary = summary_output[0]['summary_text']
        stored_summary = summary

        os.remove(audio_path)

    # Q&A
    if 'question' in request.form:
        question = request.form['question']
        if stored_transcript:
            try:
                result = qa_pipeline(question=question, context=stored_transcript)
                answer = result['answer']
            except:
                answer = "Sorry, I couldn't find an answer."

    # Translation
    if 'language' in request.form:
        lang = request.form['language']
        if stored_summary:
            try:
                translated = translator.translate(stored_summary, dest=lang).text
            except:
                translated = "Translation failed."

    return render_template_string(HTML,
                                  transcript=stored_transcript if stored_transcript else None,
                                  summary=stored_summary if stored_summary else None,
                                  answer=answer,
                                  translated=translated)

# Start Flask in background
def run_flask():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run_flask).start()
output.serve_kernel_port_as_window(5000)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 78.6MiB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Try `serve_kernel_port_as_iframe` instead. 
 * Serving Flask app '__main__'
 * Debug mode: off


<IPython.core.display.Javascript object>

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
